<a href="https://colab.research.google.com/github/Dipen2001/MY-RESUME/blob/main/fake_news_test_2_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras import Sequential
from keras.layers import Embedding, Dense, LSTM
# The keras.preprocessing.text module was removed in Keras 3.
# Use tensorflow.keras.preprocessing.text instead.
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
import nltk
from nltk.stem.snowball import SnowballStemmer
import regex as re
from nltk.tokenize import sent_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
df_fake = pd.read_csv("/content/Fake.csv")
df_true = pd.read_csv("/content/True.csv")

In [ ]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
# label them seperately
df_true['status'] = 0
df_fake['status'] = 1

In [10]:
# merge and remove unnecessary columns
df = pd.concat([df_true,df_fake])
df.drop(['subject','text','date'],axis=1,inplace=True)

In [11]:
# let's blend the smoothie
random_indexes = np.random.randint(0,len(df),len(df))
df = df.iloc[random_indexes].reset_index(drop=True)

In [12]:
pd.set_option('display.max_colwidth', 500)
random = np.random.randint(0,len(df),20)
df.iloc[random]

,title,status
10191,Senior Democratic National Committee officials resign: DNC,0
15036,"With Just 2 Words, Some Muslim Men GLORIOUSLY Exposed The ‘Christian’ Right’s Hate (SCREENSHOTS)",1
24682,Democrat Schiff says House Russia probe needs to go forward,0
15939,"LOU DOBBS Speaks Out On Trump, China And Taiwan [Video]",1
9798,WATCH: West Virginia Governor Announces He’s Leaving Dem Party At Massive Trump Rally…Switching To GOP…More #Winning,1
41457,FRIDAY DOCUMENT DUMP: 300 CLINTON E-MAILS RELEASED,1
538,CORRUPT STATE DEPARTMENT Deletes Key Info From Press Conference…Who Demanded This?,1
42928,Senate leaders look to work with White House to block Moore,0
36006,Trump: Dwayne Wade’s Cousin Was Murdered So African Americans Will Vote For Me (TWEETS),1
37993,Austria promises to consult Rome on passport offer to Italians,0


In [13]:
# Null values
df.isnull().sum()

,0
title,0
status,0


In [14]:
# longest sentence length
def longest_sentence_length(text):
  return len(text.split())

df['maximum_length'] = df['title'].apply(lambda x : longest_sentence_length(x))
print('longest sentence having length -')
max_length = max(df['maximum_length'].values)
print(max_length)

longest sentence having length -
42


In [15]:
# Text cleaning
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"

def preprocess_filter(text, stem=False):
  text = re.sub(text_cleaning, " ",str(text.lower()).strip())
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        stemmer = SnowballStemmer(language='english')
        token = stemmer.stem(token)
      tokens.append(token)
  return " ".join(tokens)

In [16]:
# Word embedding with pre padding
def one_hot_encoded(text,vocab_size=5000,max_length = 40):
    hot_encoded = one_hot(text,vocab_size)
    return hot_encoded

In [17]:
# word embedding pipeline
def word_embedding(text):
    preprocessed_text=preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

In [18]:
# Creating NN Model
embedded_features = 40
model = Sequential()
model.add(Embedding(5000,embedded_features,input_length = max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
# One hot encoded title
one_hot_encoded_title =df['title'].apply(lambda x : word_embedding(x)).values

In [20]:
# padding to make the size equal of the sequences
padded_encoded_title = pad_sequences(one_hot_encoded_title,maxlen=max_length,padding = 'pre')

In [21]:
# Splitting
X = padded_encoded_title
y = df['status'].values
y = np.array(y)

# shapes
print(X.shape)
print(y.shape)

(44898, 42)
(44898,)


In [22]:
# shape and size
print('X shape {}'.format(X.shape))
print('y shape {}'.format(y.shape))

X shape (44898, 42)
y shape (44898,)


In [23]:
# Splitting into training, testing
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 42)

# Shape and size of train and test dataset
print('X train shape {}'.format(X_train.shape))
print('X test shape {}'.format(X_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

X train shape (33673, 42)
X test shape (11225, 42)
y train shape (33673,)
y test shape (11225,)


In [24]:
# Model training
# training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 47s 81ms/step - accuracy: 0.8698 - loss: 0.3134 - val_accuracy: 0.9537 - val_loss: 0.1178
Epoch 2/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 83s 83ms/step - accuracy: 0.9764 - loss: 0.0694 - val_accuracy: 0.9650 - val_loss: 0.1024
Epoch 3/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 81s 80ms/step - accuracy: 0.9890 - loss: 0.0340 - val_accuracy: 0.9662 - val_loss: 0.1119
Epoch 4/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 46s 87ms/step - accuracy: 0.9936 - loss: 0.0211 - val_accuracy: 0.9646 - val_loss: 0.1363
Epoch 5/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 77s 77ms/step - accuracy: 0.9924 - loss: 0.0217 - val_accuracy: 0.9641 - val_loss: 0.1427
Epoch 6/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 40s 76ms/step - accuracy: 0.9962 - loss: 0.0133 - val_accuracy: 0.9681 - val_loss: 0.1619
Epoch 7/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 39s 74ms/step - accuracy: 0.9987 - loss: 0.0051 - val_accuracy: 0.9696 - val_loss: 0.1933
Epoch 8/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9992 - loss: 0.0027 - 

In [25]:
# setting threshold value
def best_threshold_value(thresholds:list,X_test):
    accuracies = []
    for thresh in thresholds:
        ypred =model.predict(X_test)
        ypred = np.where(ypred> thresh,1,0)
        accuracies.append(accuracy_score(y_test,ypred))
    return pd.DataFrame({
        'Threshold': thresholds,
        'Accuracy' : accuracies
    })

In [26]:
best_threshold_value([0.4,0.5,0.6,0.7,0.8,0.9], X_test)

351/351 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


,Threshold,Accuracy
0,0.4,0.967394
1,0.5,0.967483
2,0.6,0.967661
3,0.7,0.968196
4,0.8,0.968463
5,0.9,0.969265


In [27]:
# Predictino value at threshold 0.4
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >0.4, 1, 0)

351/351 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


In [28]:
# Confusion matrix
print('Confusion matrix')
print(confusion_matrix(y_pred,y_test))
print('----------------')
print('Classification report')
print(classification_report(y_pred,y_test))

Confusion matrix
[[5148  156]
 [ 210 5711]]
----------------
Classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5304
           1       0.97      0.96      0.97      5921

    accuracy                           0.97     11225
   macro avg       0.97      0.97      0.97     11225
weighted avg       0.97      0.97      0.97     11225



In [29]:
# input generator
def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded],maxlen=max_length,padding = 'pre')
    output = model.predict(padded_encoded_title)
    output = np.where(0.4>output,1,0)
    if output[0][0] == 1:
        return 'Yes this News is fake'
    return 'No, It is not fake'

In [30]:
# predictions
prediction_input_processing('Americans are more concerned over Indians fake open source contribution')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


'Yes this News is fake'

In [31]:
news = 'Trump Just Sent Michelle Obama a Bill She will Never Be able to pay in her lifetime'
prediction_input_processing(news)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


'No, It is not fake'

In [41]:
!pip install flask-ngrok

In [44]:
# Assuming you have already defined and compiled your model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Epoch 1/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 42s 80ms/step - accuracy: 1.0000 - loss: 1.3484e-04 - val_accuracy: 0.9687 - val_loss: 0.2630
Epoch 2/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 81s 79ms/step - accuracy: 1.0000 - loss: 5.3487e-05 - val_accuracy: 0.9685 - val_loss: 0.2778
Epoch 3/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - accuracy: 1.0000 - loss: 4.0221e-05 - val_accuracy: 0.9690 - val_loss: 0.2880
Epoch 4/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 80s 75ms/step - accuracy: 1.0000 - loss: 2.3861e-05 - val_accuracy: 0.9686 - val_loss: 0.2980
Epoch 5/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 1.0000 - loss: 1.6101e-05 - val_accuracy: 0.9684 - val_loss: 0.3063
Epoch 6/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 1.0000 - loss: 8.1415e-06 - val_accuracy: 0.9683 - val_loss: 0.3160
Epoch 7/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 1.0000 - loss: 6.4499e-06 - val_accuracy: 0.9682 - val_loss: 0.3235
Epoch 8/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 42s 79ms/step - accura

In [45]:
model.save('fake_news_model.h5')

In [46]:
from google.colab import files
files.download('fake_news_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
from keras.models import load_model

# Load the model from the correct path
model = load_model('/content/fake_news_model.h5')  # Adjust the path as needed

In [48]:
from google.colab import files
uploaded = files.upload()

Saving fake_news_model.h5 to fake_news_model (1).h5


In [51]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Load your pre-trained model
from keras.models import load_model
model = load_model('fake_news_model.h5')  # Replace with your model path

# Define text cleaning and preprocessing functions
stop_words = stopwords.words('english')
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"
stemmer = SnowballStemmer(language='english')

def preprocess_filter(text, stem=False):
    text = re.sub(text_cleaning, " ", str(text.lower()).strip())
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                token = stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

def one_hot_encoded(text, vocab_size=5000):
    hot_encoded = one_hot(text, vocab_size)
    return hot_encoded

def word_embedding(text):
    preprocessed_text = preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

max_length = 40  # Replace with your max_length value

# Define the prediction function
def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded], maxlen=max_length, padding='pre')
    output = model.predict(padded_encoded_title)
    output = np.where(output > 0.4, 1, 0)
    if output[0][0] == 1:
        return 'Yes, this news is fake.'
    return 'No, it is not fake.'

# Define the route for the home page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        news_text = request.form['news_text']
        result = prediction_input_processing(news_text)
        return render_template_string('''
            <h1>Fake News Detection</h1>
            <form method="POST">
                <textarea name="news_text" rows="4" cols="50" placeholder="Enter news text here...">{{ news_text if news_text else '' }}</textarea><br><br>
                <input type="submit" value="Check">
            </form>
            {% if result %}
            <h2>Result:</h2>
            <p>{{ result }}</p>
            {% endif %}
        ''', result=result, news_text=news_text)
    return render_template_string('''
        <h1>Fake News Detection</h1>
        <form method="POST">
            <textarea name="news_text" rows="4" cols="50" placeholder="Enter news text here..."></textarea><br><br>
            <input type="submit" value="Check">
        </form>
    ''')

# Run the Flask app
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [1]:
!pip uninstall -y flask-ngrok pyngrok
!pip install flask-ngrok pyngrok

Found existing installation: flask-ngrok 0.0.25
Uninstalling flask-ngrok-0.0.25:
  Successfully uninstalled flask-ngrok-0.0.25
Found existing installation: pyngrok 7.2.3
Uninstalling pyngrok-7.2.3:
  Successfully uninstalled pyngrok-7.2.3
  Using cached flask_ngrok-0.0.25-py3-none-any.whl.metadata (1.8 kB)
  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)


In [1]:
!pip uninstall -y flask-ngrok pyngrok
!pip install flask-ngrok pyngrok

Found existing installation: flask-ngrok 0.0.25
Uninstalling flask-ngrok-0.0.25:
  Successfully uninstalled flask-ngrok-0.0.25
Found existing installation: pyngrok 7.2.3
Uninstalling pyngrok-7.2.3:
  Successfully uninstalled pyngrok-7.2.3
  Using cached flask_ngrok-0.0.25-py3-none-any.whl.metadata (1.8 kB)
  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)


In [2]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Load your pre-trained model
from keras.models import load_model
model = load_model('/content/fake_news_model (1).h5')  # Replace with your model path

# Define text cleaning and preprocessing functions
stop_words = stopwords.words('english')
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"
stemmer = SnowballStemmer(language='english')

def preprocess_filter(text, stem=False):
    text = re.sub(text_cleaning, " ", str(text.lower()).strip())
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                token = stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

def one_hot_encoded(text, vocab_size=5000):
    hot_encoded = one_hot(text, vocab_size)
    return hot_encoded

def word_embedding(text):
    preprocessed_text = preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

max_length = 40  # Replace with your max_length value

# Define the prediction function
def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded], maxlen=max_length, padding='pre')
    output = model.predict(padded_encoded_title)
    output = np.where(output > 0.4, 1, 0)
    if output[0][0] == 1:
        return 'Yes, this news is fake.'
    return 'No, it is not fake.'

# Define the route for the home page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        news_text = request.form['news_text']
        result = prediction_input_processing(news_text)
        return render_template_string('''
            <h1>Fake News Detection</h1>
            <form method="POST">
                <textarea name="news_text" rows="4" cols="50" placeholder="Enter news text here...">{{ news_text if news_text else '' }}</textarea><br><br>
                <input type="submit" value="Check">
            </form>
            {% if result %}
            <h2>Result:</h2>
            <p>{{ result }}</p>
            {% endif %}
        ''', result=result, news_text=news_text)
    return render_template_string('''
        <h1>Fake News Detection</h1>
        <form method="POST">
            <textarea name="news_text" rows="4" cols="50" placeholder="Enter news text here..."></textarea><br><br>
            <input type="submit" value="Check">
        </form>
    ''')

# Run the Flask app
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [10]:
!pip install pyngrok
!ngrok authtoken 2tkOKPW0nmjoFrQTAbLD9cOvQf4_6LMkZY2bNfcsAM8FeWN74  # Get it from https://dashboard.ngrok.com/get-started
!ngrok http 5000


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Traceback (most recent call last):
  File "/usr/lib/python3.11/subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 2053, in _wait
    (pid, sts) = self._try_wait(0)
                 ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 2011, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 617, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 605, in run
    process.run_process(pyngrok_config.ngrok_pa

SyntaxError: invalid decimal literal (<ipython-input-9-786af7df2a65>, line 1)